#      <center> COVID-19 Data Analysis: Algeria and the World </center>

### <center> Singhealth Singapore General Hospital Take Home Exercise </center>
### <center> Ng Guangren, Ryan </center>

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

from sklearn.metrics.pairwise import nan_euclidean_distances

import sqlite3


In [28]:
df = pd.read_csv(r"Data.csv")


# Data Exploration

In [29]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

Turning object type features into string and datetime respectively helps in narrowingg down the scope. For example numbers cannot be placed in a string feature. It also allows data type specific operations.[<sup>1</sup>](https://towardsdatascience.com/why-we-need-to-use-pandas-new-string-dtype-instead-of-object-for-textual-data-6fd419842e24)

In [30]:
object_to_string = {
    'Entity': 'string',
    'Continent': 'string',
}

df = df.astype(object_to_string)
df["Date"] = pd.to_datetime(df.Date)


In [31]:
df = df.round(3)
df


,Entity,Continent,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Date,Daily tests,Cases,Deaths
0,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-25,8.0,NaN,NaN
1,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-26,5.0,NaN,NaN
2,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-27,4.0,NaN,NaN
3,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-28,1.0,NaN,NaN
4,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-29,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38467,Zimbabwe,Africa,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2021-02-24,1804.0,35960.0,1456.0
38468,Zimbabwe,Africa,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2021-02-25,2965.0,35994.0,1458.0
38469,Zimbabwe,Africa,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2021-02-26,NaN,36044.0,1463.0
38470,Zimbabwe,Africa,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2021-02-27,NaN,36058.0,1463.0


In [32]:
df.Entity.unique()


<StringArray>
[             'Albania',              'Algeria',            'Argentina',
              'Armenia',            'Australia',              'Austria',
              'Bahrain',           'Bangladesh',              'Belarus',
              'Belgium',
 ...
               'Turkey',               'Uganda',              'Ukraine',
 'United Arab Emirates',       'United Kingdom',        'United States',
              'Uruguay',              'Vietnam',               'Zambia',
             'Zimbabwe']
Length: 104, dtype: string

The word "Entity" is a little too generalised. I will consider them as countries as it is synonmous as sovereign states. 

In [33]:
df.rename({"Entity": "Country", "GDP/Capita": "GDP per capita"},
          inplace=True, axis=1)


In [34]:
df.isnull().sum()


Country                               0
Continent                             0
Latitude                              0
Longitude                             0
Average temperature per year          0
Hospital beds per 1000 people         0
Medical doctors per 1000 people       0
GDP per capita                        0
Population                            0
Median age                            0
Population aged 65 and over (%)       0
Date                                  0
Daily tests                        7895
Cases                               254
Deaths                             3610
dtype: int64

In [35]:
# df.groupby('Country').mean(numeric_only=True)[["Daily tests","Cases","Deaths"]].sort_values(["Daily tests","Cases","Deaths"]).head(20)


In [36]:
df.groupby('Country').mean()[["Daily tests", "Cases", "Deaths"]].isnull().any()


Daily tests    False
Cases          False
Deaths         False
dtype: bool

This cell above shows that after grouping by country, each country has at least one numerical value so calculating mean doesn't not return null.

In [70]:
# imputer =KNNImputer(n_neighbors=1)
# imputer.fit_transform(df[["Daily tests", "Cases", "Deaths"]])

In [58]:

# print(df.index)

# for name, group in df.groupby('Country'):
#     nan_e_dist = nan_euclidean_distances(group[["Daily tests"]])
#     num = list(filter(lambda x: x > 0, sorted(nan_e_dist[0])))[0]
#     print(type(group))
#     imputer = KNNImputer(n_neighbors=num)
#     # print(df[df.Country == name][["Daily tests", "Cases", "Deaths"]])
#     print(group[["Daily tests", "Cases", "Deaths"]].index)
#     t = group[["Daily tests", "Cases", "Deaths"]]
#     # t.reset_index(inplace=True)
#     t = pd.DataFrame(t)
#     print(t)
#     x = imputer.fit_transform(X=t)
#     print(x)
#     break

# imputer = KNNImputer(n_neighbors=1)


https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/

In [68]:
imputer = SimpleImputer()

df[["Daily tests", "Cases", "Deaths"]] =imputer.fit_transform(df[["Daily tests", "Cases", "Deaths"]])

df.isnull().any()

Country                            False
Continent                          False
Latitude                           False
Longitude                          False
Average temperature per year       False
Hospital beds per 1000 people      False
Medical doctors per 1000 people    False
GDP per capita                     False
Population                         False
Median age                         False
Population aged 65 and over (%)    False
Date                               False
Daily tests                        False
Cases                              False
Deaths                             False
dtype: bool

In [5]:
# df = df.groupby('Country').apply(lambda x: x.fillna(method='ffill'))
# df = df.groupby('Country').apply(lambda x: x.fillna(method='bfill'))
# df = df[df.Cases > 0]


# 5- Statistical description of the data

In [69]:
df.describe().round(2)


,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP per capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
count,38472.00,38472.00,38472.00,38472.00,38472.00,38472.00,3.847200e+04,38472.00,38472.00,38472.00,38472.00,38472.00
mean,23.74,20.21,17.72,3.17,2.09,19002.33,4.896983e+07,32.75,10.66,39440.59,287902.66,8090.50
std,26.06,61.07,8.13,2.56,1.52,22271.11,1.427251e+08,8.47,6.77,133890.32,1400596.22,28128.23
min,-40.90,-106.35,-2.00,0.20,0.02,411.60,3.412840e+05,16.00,1.00,-239172.00,1.00,1.00
25%,8.62,-3.44,11.00,1.40,0.82,3659.00,4.793900e+06,27.00,5.00,2244.75,2101.00,103.00
50%,27.51,21.82,20.00,2.50,1.89,8821.80,1.148464e+07,32.00,8.00,11055.00,22246.50,766.00
75%,45.94,47.48,25.00,4.49,3.21,25946.20,4.286296e+07,41.00,16.00,39440.59,141121.75,7539.25
max,64.96,179.41,29.00,13.05,7.52,114704.60,1.339180e+09,48.00,28.00,2945871.00,28605669.00,513091.00


# Sqlite3 Setup, creations and insertions

# 6- Visualize the data